In [1]:
import pandas as pd
import numpy as np
import collections
import re
import sqlite3

In [2]:
!pwd

/Users/nathan.cheever/OneDrive - AdvancedMD/ds/open/meetings_trainings_talks/tds/python_trainings


In [3]:
# Change this to your path to the DB file!
conn = sqlite3.connect("/Users/nathan.cheever/Documents/gitlab/coaching/flights/flights.db")

### Load raw data 

In [4]:
# load our datasets
# These first four are used as lookup tables
airlines =     pd.read_sql('SELECT * FROM airlines', con=conn)
airports =     pd.read_sql('SELECT * FROM airports', con=conn)
cancel_codes = pd.read_sql('SELECT * FROM cancel_codes', con=conn)
carriers =     pd.read_sql('SELECT * FROM carriers', con=conn)
# This has all the data about flights
flights =      pd.read_sql('SELECT * FROM flights', con=conn)

In [5]:
#Closing the connection to the db, since we're done using it
conn.close()

### Some transforms to get it ready for analysis

In [6]:
# Turn our flights table into a list of dicts
# Normally we wouldn't do this, but for the sake of learning to deal with data
# structures in Python, we'll do it this way :)
flights = flights.to_dict(orient='records')

### HW 1: Answer the following questions about the `flights` :

What datatype is our `flights` object now (a list, dict, etc)?

How many elements are in it?

How can we look at/inspect the first element?

### HW 2: Create dictionaries from our lookup tables, making the `Code` the key and the `Description` the value

In [7]:
cancel_codes_dict = dict(zip(cancel_codes.Code, cancel_codes.Description))

# Go ahead and create more dictionaries for airlines, airports and carriers where
# for each the key will be the `Code` and the value will be the `Description`
airlines_dict = dict(zip(airlines.Code, airlines.Description))

airports_dict = dict(zip(airports.Code, airports.Description))

carriers_dict = dict(zip(carriers.Code, carriers.Description))

# When you're done there should be four dicts (cancel_codes_dict, airlines_dict, airports_dict, carriers_dict)

In [8]:
flights[0]

{'DAY_OF_MONTH': 1,
 'DAY_OF_WEEK': 3,
 'FL_DATE': '2017-02-01',
 'UNIQUE_CARRIER': 'B6',
 'AIRLINE_ID': 20409,
 'FL_NUM': 33,
 'ORIGIN_AIRPORT_ID': 10785,
 'ORIGIN_CITY_NAME': 'Burlington, VT',
 'ORIGIN_STATE_NM': 'Vermont',
 'DEST_AIRPORT_ID': 12478,
 'DEST_CITY_NAME': 'New York, NY',
 'DEST_STATE_NM': 'New York',
 'DEP_TIME': 1857.0,
 'DEP_DELAY': -10.0,
 'TAXI_OUT': 16.0,
 'WHEELS_OFF': 1913.0,
 'WHEELS_ON': 2019.0,
 'TAXI_IN': 10.0,
 'ARR_TIME': 2029.0,
 'ARR_DELAY': -8.0,
 'CANCELLED': 0,
 'CANCELLATION_CODE': None,
 'DIVERTED': 0,
 'ACTUAL_ELAPSED_TIME': 92.0,
 'FLIGHTS': 1,
 'DISTANCE': 266,
 'CARRIER_DELAY': nan,
 'WEATHER_DELAY': nan,
 'NAS_DELAY': nan,
 'SECURITY_DELAY': nan,
 'LATE_AIRCRAFT_DELAY': nan,
 'Unnamed: 31': None}

### Lists

In [11]:
flights[:3]

[{'DAY_OF_MONTH': 1,
  'DAY_OF_WEEK': 3,
  'FL_DATE': '2017-02-01',
  'UNIQUE_CARRIER': 'B6',
  'AIRLINE_ID': 20409,
  'FL_NUM': 33,
  'ORIGIN_AIRPORT_ID': 10785,
  'ORIGIN_CITY_NAME': 'Burlington, VT',
  'ORIGIN_STATE_NM': 'Vermont',
  'DEST_AIRPORT_ID': 12478,
  'DEST_CITY_NAME': 'New York, NY',
  'DEST_STATE_NM': 'New York',
  'DEP_TIME': 1857.0,
  'DEP_DELAY': -10.0,
  'TAXI_OUT': 16.0,
  'WHEELS_OFF': 1913.0,
  'WHEELS_ON': 2019.0,
  'TAXI_IN': 10.0,
  'ARR_TIME': 2029.0,
  'ARR_DELAY': -8.0,
  'CANCELLED': 0,
  'CANCELLATION_CODE': None,
  'DIVERTED': 0,
  'ACTUAL_ELAPSED_TIME': 92.0,
  'FLIGHTS': 1,
  'DISTANCE': 266,
  'CARRIER_DELAY': nan,
  'WEATHER_DELAY': nan,
  'NAS_DELAY': nan,
  'SECURITY_DELAY': nan,
  'LATE_AIRCRAFT_DELAY': nan,
  'Unnamed: 31': None},
 {'DAY_OF_MONTH': 1,
  'DAY_OF_WEEK': 3,
  'FL_DATE': '2017-02-01',
  'UNIQUE_CARRIER': 'B6',
  'AIRLINE_ID': 20409,
  'FL_NUM': 34,
  'ORIGIN_AIRPORT_ID': 12478,
  'ORIGIN_CITY_NAME': 'New York, NY',
  'ORIGIN_STATE_NM'

In [9]:
my_list = [1, 'a', pd.DataFrame(), [1, 2, 3], {}, ]

In [12]:
my_list.append('Jake')

In [14]:
my_list.reverse()

In [15]:
my_list

['Jake',
 {},
 [1, 2, 3],
 Empty DataFrame
 Columns: []
 Index: [],
 'a',
 1]

The one downside of lists... _n_ elements: O(n)

In [16]:
len(flights)

410517

In [18]:
flights[999]

{'DAY_OF_MONTH': 1,
 'DAY_OF_WEEK': 3,
 'FL_DATE': '2017-02-01',
 'UNIQUE_CARRIER': 'DL',
 'AIRLINE_ID': 19790,
 'FL_NUM': 763,
 'ORIGIN_AIRPORT_ID': 12323,
 'ORIGIN_CITY_NAME': 'Wilmington, NC',
 'ORIGIN_STATE_NM': 'North Carolina',
 'DEST_AIRPORT_ID': 10397,
 'DEST_CITY_NAME': 'Atlanta, GA',
 'DEST_STATE_NM': 'Georgia',
 'DEP_TIME': 707.0,
 'DEP_DELAY': -8.0,
 'TAXI_OUT': 16.0,
 'WHEELS_OFF': 723.0,
 'WHEELS_ON': 826.0,
 'TAXI_IN': 10.0,
 'ARR_TIME': 836.0,
 'ARR_DELAY': -23.0,
 'CANCELLED': 0,
 'CANCELLATION_CODE': None,
 'DIVERTED': 0,
 'ACTUAL_ELAPSED_TIME': 89.0,
 'FLIGHTS': 1,
 'DISTANCE': 377,
 'CARRIER_DELAY': nan,
 'WEATHER_DELAY': nan,
 'NAS_DELAY': nan,
 'SECURITY_DELAY': nan,
 'LATE_AIRCRAFT_DELAY': nan,
 'Unnamed: 31': None}

In [7]:
cancel_codes_dict = dict(zip(cancel_codes.Code, cancel_codes.Description))

# Go ahead and create more dictionaries for airlines, airports and carriers where
# for each the key will be the `Code` and the value will be the `Description`
airlines_dict = dict(zip(airlines.Code, airlines.Description))
airports_dict = dict(zip(airports.Code, airports.Description))
carriers_dict = dict(zip(carriers.Code, carriers.Description))

# When you're done there should be four dicts (cancel_codes_dict, airlines_dict, airports_dict, carriers_dict)

In [19]:
cancel_codes_dict

{'A': 'Carrier', 'B': 'Weather', 'C': 'National Air System', 'D': 'Security'}

In [29]:
cancel_codes_dict['E']

KeyError: 'E'

In [30]:
cancel_codes_dict.get('E', 'missing!')

'missing!'

### HW 3: Add in our dictionary mappings to our `flights` data
Using our mappings, please create the following new keys in each 'row' of flights with the appropriate value:
 - AIRLINE_NAME
 - ORIGIN_AIRPORT_NAME
 - CARRIER_NAME
 - CANCELLATION_REASON

In [35]:
flights[0]['UNIQUE_CARRIER']

'B6'

In [38]:
carriers_dict['B6']

'JetBlue Airways'

In [44]:
for row_dict in flights:
    # Here we match codes to loop up description values from dictionaries
    row_dict['AIRLINE_NAME'] = airlines_dict.get(row_dict['AIRLINE_ID'])
    row_dict['CARRIER_NAME'] = carriers_dict.get(row_dict['UNIQUE_CARRIER'])
    row_dict['ORIGIN_AIRPORT_NAME'] = airports_dict.get(row_dict['ORIGIN_AIRPORT_ID'])
    row_dict['CANCELLATION_REASON'] = cancel_codes_dict.get(row_dict['CANCELLATION_CODE'], 'No reason')
    
# Here I use the .get() method for key value lookups. It's a safe bet, especially on cancel_codes_dict, since not all flights
# were cancelled. If you use the cancel_codes_dict[row_dict['CANCELLATION_CODE']] and that flight dict has None as it's value for 'CANCELLATION_CODE', 
# then it's equivalent to cancel_codes_dict[ None ], and since None isn't a key in that dict, you'll get a program-halting "KeyError".

# With the .get() method, if the key isn't found in your dict, it will return None, or a specified default value. I've set mine to be 'No reason'.

### Question 1: What are the unique airlines? 
Using the `AIRLINE_NAME` field

In [71]:
# We can use a set comprehension
{ row['AIRLINE_NAME'] for row in flights}

{'Alaska Airlines Inc.: AS',
 'American Airlines Inc.: AA',
 'Delta Air Lines Inc.: DL',
 'ExpressJet Airlines Inc.: EV',
 'Frontier Airlines Inc.: F9',
 'Hawaiian Airlines Inc.: HA',
 'JetBlue Airways: B6',
 'SkyWest Airlines Inc.: OO',
 'Southwest Airlines Co.: WN',
 'Spirit Air Lines: NK',
 'United Air Lines Inc.: UA',
 'Virgin America: VX'}

### Question 2: Which flight had the longest departure delay?

In [69]:
%%timeit 

max_delay = 0
flight_ = None

for i in flights:
#     # Version 1: two lookups of i['DEP_DELAY']
#     if i['DEP_DELAY'] > max_delay:
#         max_delay = i['DEP_DELAY']
#         flight_ = i
        
#     # Version 2: variable assignment
#     dep_delay = i['DEP_DELAY']
#     if dep_delay > max_delay:
#         max_delay = dep_delay
#         flight_ = i
    
    # Version 3: Walrus Operator!
    if (dep_delay := i['DEP_DELAY']) > max_delay:
        max_delay = dep_delay
        flight_ = i

82.3 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
%%timeit
# Here's a one liner
max(flights, key=lambda d: d['DEP_DELAY'])

109 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Question 3: Which carrier had the most time delay due to weather? What are the top 10?